# Driver Fatigue Detection using CNN

Implemented a Convolutional Neural Network (CNN) to classify eyes as either open or closed, also employed facial recognition techniques to identify eye coordinates in real time.


## Setup the environment

In [ ]:
!pip install tensorflow scikit-learn opencv-python numpy matplotlib face_recognition

---

## Load images for training

The `load_eye_images` function is employed to read images from a target folder, resize them to a consistent size (80x80 pixels), and assign labels based on the provided binary flag for eye state (0 for open, 1 for closed).



In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_eye_images(source_path, eye_state=0):
    """
    Returns:
        list: List of eye image data with corresponding labels.
    """
    image_count = 0
    error_count = 0
    eye_images = []

    for filename in os.listdir(source_path):
        try:
            img = cv2.imread(os.path.join(source_path, filename))
            img = cv2.resize(img, (80, 80))
            eye_images.append([img, eye_state])
        except Exception as e:
            error_count += 1
            print(f'Error Count: {error_count}, Exception: {e}')
            continue

        image_count += 1
        if image_count % 500 == 0:
            print(f'Successful Image Import Count: {image_count}')

    return eye_images


---

## Prepare the data for training

Now, we need to organize and prepare the image data for model training. Images from folders representing open and closed eyes are loaded using the `load_eye_images` function. The data is then converted into NumPy arrays, normalized, and split into training and testing sets.

### Eye open

In [ ]:
open_eye_folder = "../train/OpenEye"
open_eye_data = load_eye_images(open_eye_folder, eye_state=0)

# sample
img, label = open_eye_data[20]
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

### Eye closed

In [ ]:
closed_eye_folder = "../train/ClosedEye"
closed_eye_data = load_eye_images(closed_eye_folder, eye_state=1)

# sample
img, label = closed_eye_data[10]
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
eye_data = closed_eye_data + open_eye_data

X = [] 
y = [] 
for features, label in eye_data: 
     X.append(features)
     y.append(label)

# Prepare data
X = np.array(X).reshape(-1, 80, 80, 3) / 255.0
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)


---

## Build a Convolutional Neural Network (CNN) model

We need to build a Convolutional Neural Network (CNN) Model for drowsiness detection. The model consists of convolutional layers, max-pooling layers, and dense layers. It is compiled using the Adam optimizer and binary crossentropy loss function.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout

In [ ]:
# Build the CNN model
cnn_model = Sequential()

cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 3)))
cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC(curve='PR')])


---

## Model Training and Evaluation with Accuracy Plotting

The fit method is used to train the model with specified batch size, validation data, and number of epochs. The evaluate method calculates the loss and specified metrics on the test data.

### Train the model

In [ ]:
history = cnn_model.fit(X_train, y_train, batch_size=800, validation_data=(X_test, y_test), epochs=24, verbose=1)

### Evaluate the model

In [ ]:
cnn_model.evaluate(X_test, y_test, verbose=1)

### Plot the accuracy

In [ ]:
training_accuracy = history.history['auc']
validation_accuracy = history.history['val_auc']

# Plot accuracy over epochs
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy Over Epochs')
plt.legend()
plt.show()

### Save the model

In [ ]:
cnn_model.save('drowsiness_model.keras')


---

## Alert system

Frames from the webcam are processed in real-time, and the model predicts the state of the eyes. The loop also handles visual feedback and alerts based on the prediction


### Load trained model

In [ ]:
trained_model = keras.models.load_model('drowsiness_model.keras')


### Extract eye region for prediction

The eyes are cropped from a webcam frame. It utilizes the `face_recognition` library to detect facial features and accurately crop the eyes.

In [ ]:
import face_recognition

def extract_eye_region_for_prediction(frame):

    # Detect facial landmarks
    facial_landmarks_list = face_recognition.face_landmarks(frame)

    if not facial_landmarks_list:
        return

    # Get left and right eye coordinates
    try:
        left_eye_coordinates = facial_landmarks_list[0]['left_eye']
        right_eye_coordinates = facial_landmarks_list[0]['right_eye']
    except KeyError:
        return

    # Draw rectangles around each eye
    for eye_coordinates in [left_eye_coordinates, right_eye_coordinates]:
        x_max = max([x for x, y in eye_coordinates])
        x_min = min([x for x, y in eye_coordinates])
        y_max = max([y for x, y in eye_coordinates])
        y_min = min([y for x, y in eye_coordinates])

        # Calculate the range of x and y coordinates
        x_range = x_max - x_min
        y_range = y_max - y_min

        # Ensure the full eye is captured by calculating coordinates of a square with a 50% cushion
        if x_range > y_range:
            right_bound = round(0.5 * x_range) + x_max
            left_bound = x_min - round(0.5 * x_range)
            bottom_bound = round((right_bound - left_bound - y_range) / 2) + y_max
            top_bound = y_min - round((right_bound - left_bound - y_range) / 2)
        else:
            bottom_bound = round(0.5 * y_range) + y_max
            top_bound = y_min - round(0.5 * y_range)
            right_bound = round((bottom_bound - top_bound - x_range) / 2) + x_max
            left_bound = x_min - round((bottom_bound - top_bound - x_range) / 2)

        # Draw rectangle around the eye
        cv2.rectangle(frame, (left_bound, top_bound), (right_bound, bottom_bound), (255, 0, 0), 2)

    # Crop the image according to the determined coordinates
    cropped_eye_region = frame[top_bound:bottom_bound + 1, left_bound:right_bound + 1]

    # Resize the cropped image to 80x80 pixels
    resized_cropped_eye_region = cv2.resize(cropped_eye_region, (80, 80))

    # Reshape the image for model prediction
    image_for_prediction = resized_cropped_eye_region.reshape(-1, 80, 80, 3)

    return image_for_prediction

### Initialize the webcam

In [ ]:
def initialize_webcam():
    cap = cv2.VideoCapture(0)
    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    
    if not cap.isOpened():
        raise IOError('Cannot open webcam')

    return cap, w, h


### Run drowsiness detection

It uses computer vision to detect drowsiness from a webcam feed. It continuously captures and processes frames, extracting the eye region for analysis. A pre-trained model predicts eye states, triggering a visual alert for prolonged eye closure, indicating potential drowsiness. (*App can be closed by pressing 'esc'*)

In [ ]:
# Initialize webcam
webcam, width, height = initialize_webcam()

# Initialize counters
frame_count = 0
blink_counter = 0

# Run a continuous loop while the webcam is active
while True:
    # Capture frames from the webcam
    ret, frame = webcam.read()

    # Use only every other frame to manage speed and memory usage
    if frame_count == 0:
        frame_count += 1
        pass
    else:
        frame_count = 0
        continue

    # Process the frame to get the eye for prediction
    eye_image = extract_eye_region_for_prediction(frame)
    try:
        eye_image = eye_image / 255.0
    except:
        continue

    # Get prediction from the trained model
    prediction = trained_model.predict(eye_image)

    # Display status based on the prediction ("Open Eyes" or "Closed Eyes")
    if prediction < 0.5:
        blink_counter = 0
        status = 'Open'
        
    else:
        blink_counter += 1
        status = 'Closed'

        # If the blink counter exceeds 2, show an alert for drowsiness
        if blink_counter > 2:
            # Show alert message on Screen
            cv2.rectangle(frame, (round(width / 2) - 160, round(height) - 200), (round(width / 2) + 175, round(height) - 120), (0, 0, 255), -1)
            cv2.putText(frame, "Blink, Don't Sleep!", (round(width / 2) - 136, round(height) - 146), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_4)
            cv2.imshow('Drowsiness Detection', frame)
            k = cv2.waitKey(1)
            blink_counter = 1
            continue

    # Show prediction on Screen
    cv2.putText(frame, f'{prediction}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
    # Show status (Open or Closed on Screen)
    cv2.putText(frame, 'Status: ' + status, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
    # Display the processed frame
    cv2.imshow('Drowsiness Detection', frame)
    
    # Exit the loop on pressing the 'Esc' key
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the webcam and close all windows
webcam.release()
cv2.destroyAllWindows()
